### Importing the required libraries and dataset

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
data=pd.read_excel(r"D:\Elahe\MSF\Thesis\Datasets\sample.xlsx")

#### Calculating daily return and shorting daily return for bull, bear and index

In [3]:
def daily_return (price, ticker):
    daily_return = ["."]
    for i in range (1, len(price)):
        if ticker[i]==ticker[i-1]:
            daily_return.append((price[i]-price[i-1])/price[i-1])
        else:
            daily_return.append(".")
    return daily_return

In [4]:
def short_daily_return (daily_return, cost):
    daily_cost=(cost+1)**(1/254)-1
    short_return=[]
    for i in range (len(daily_return)):
        if daily_return[i] != ".":
            short_return.append((daily_return[i]*-1)-daily_cost)
        else:
            short_return.append(".")
    return short_return

#### Calculating cumulative return

In [5]:
def Cumulative_Return (daily_return, period):
    Cumulative_Return=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Cumulative_Return.append(".")
        elif daily_return[i-1]==".":
            Cumulative_Return.append(daily_return[i]-1)
        elif period[i-1]==period[i]:
            Cumulative_Return.append(Cumulative_Return[i-1]*(daily_return[i]-1))
        else:
            Cumulative_Return.append(daily_return[i]-1)
    for j in range (len(Cumulative_Return)):
        if Cumulative_Return[j]!=".":
            Cumulative_Return[j]=1-abs(Cumulative_Return[j])
    return Cumulative_Return

#### Calculating sharpe ratio

In [6]:
def Sharpe_Ratio (daily_return, Rf, period):
    Partial_Data=[]
    Sharpe_Ratio=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Sharpe_Ratio.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sharpe_Ratio.append(np.mean(Partial_Data)/np.std(Partial_Data))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sharpe_Ratio.append(".")
        else:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sharpe_Ratio.append(np.mean(Partial_Data)/np.std(Partial_Data))
            Partial_Data=[]
    return Sharpe_Ratio

#### Calculating Sortino ratio

In [7]:
def Sortino_Ratio (daily_return, Rf, period):
    Partial_Data=[]
    negative_return=[]
    Sortino_Ratio=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Sortino_Ratio.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i]-Rf[i])
            if daily_return[i]-Rf[i]<0:
                negative_return.append((daily_return[i]-Rf[i])**2)
            Sortino_Ratio.append(np.mean(Partial_Data)/((np.sum(negative_return)/len(negative_return))**(1/2)))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i]-Rf[i])
            Sortino_Ratio.append(".")
            if daily_return[i]-Rf[i]<0:
                negative_return.append((daily_return[i]-Rf[i])**2)
        else:
            Partial_Data.append(daily_return[i]-Rf[i])
            if daily_return[i]-Rf[i]<0:
                negative_return.append((daily_return[i]-Rf[i])**2)
            Sortino_Ratio.append(np.mean(Partial_Data)/((np.sum(negative_return)/len(negative_return))**(1/2)))
            Partial_Data=[]
            negative_return=[]
    return Sortino_Ratio

#### Calculating analytical VaR

In [8]:
def Analytical_VaR (daily_return, period):
    Partial_Data=[]
    Analytical_VaR=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Analytical_VaR.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Analytical_VaR.append(np.mean(Partial_Data)-(1.65*np.std(Partial_Data)))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            Analytical_VaR.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Analytical_VaR.append(np.mean(Partial_Data)-(1.65*np.std(Partial_Data)))
            Partial_Data=[]
    return Analytical_VaR

#### Calculating VaR

In [9]:
def VaR (daily_return, period):
    Partial_Data=[]
    VaR=[]
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            VaR.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            VaR.append(Partial_Data[round(0.05*len(Partial_Data))-1])
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            VaR.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            VaR.append(Partial_Data[round(0.05*len(Partial_Data))-1])
            Partial_Data=[]
    return VaR

#### Calculating expected VaR

In [10]:
def Expected_VaR (daily_return, period):
    Partial_Data=[]
    Expected_VaR=[]
    Sum_Left_Tail=0
    for i in range (len(daily_return)):
        if daily_return[i]==".":
            Expected_VaR.append(".")
        elif i == len(daily_return)-1:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            for j in range(round(0.05*len(Partial_Data))):
                Sum_Left_Tail+=Partial_Data[j]
            Expected_VaR.append(Sum_Left_Tail/round(0.05*len(Partial_Data)))
        elif period[i]==period[i+1]:
            Partial_Data.append(daily_return[i])
            Expected_VaR.append(".")
        else:
            Partial_Data.append(daily_return[i])
            Partial_Data.sort()
            for j in range(round(0.05*len(Partial_Data))):
                Sum_Left_Tail+=Partial_Data[j]
            Expected_VaR.append(Sum_Left_Tail/round(0.05*len(Partial_Data)))
            Partial_Data=[]
            Sum_Left_Tail=0
    return Expected_VaR

#### Creating portfolios and finding the best strategy

In [11]:
def Portfolio (Bull_Return, Bear_Return, Bull_Weight, Bear_Weight):
    portfolio_return=[]
    for i in range (len(Bull_Return)):
        if Bull_Return[i] != ".":
            portfolio_return.append((Bull_Return[i]*Bull_Weight)+(Bear_Return[i]*Bear_Weight))
        else:
            portfolio_return.append(".")
    return portfolio_return

In [12]:
def Best_Strategy (P1,P2,P3,P4,P5,P6,P7):
    Highest_Return=[]
    Best_Strategy=[]
    for i in range (len(P1)):
        if P1[i]!=".":
            Highest_Return.append(max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]))
            if max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P1[i]:
                Best_Strategy.append("1:0")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P2[i]:
                Best_Strategy.append("0.75:0.25")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P3[i]:
                Best_Strategy.append("0.67:0.33")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P4[i]:
                Best_Strategy.append("0.5:0.5")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P5[i]:
                Best_Strategy.append("0.33:0.67")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P6[i]:
                Best_Strategy.append("0.25:0.75")
            elif max(P1[i],P2[i],P3[i],P4[i],P5[i],P6[i],P7[i]) == P7[i]:
                Best_Strategy.append("0:1")
        else:
            Highest_Return.append(".")
            Best_Strategy.append(".")
    return Highest_Return, Best_Strategy

### Removing empty observations

In [13]:
def Remove_Character (dataset, list_name, Character):
    removal_rows=[]
    for i in range (len(list_name)):
        if list_name[i]==Character:
            removal_rows.append(i)
    dataset=dataset.drop(removal_rows)
    dataset=dataset.reset_index()
    return dataset

### Defining a main function to call all functions at the same time

In [14]:
def Main_Function (data, cost, period):
    
    # Calculating daily returns
    data["Bull Return"]=daily_return(data["Bull Price"],data["Code"])
    data["Bear Return"]=daily_return(data["Bear Price"],data["Code"])
    data["Index Return"]=daily_return(data["Index Price"],data["Code"])
    
    # Calculating shorting daily returns
    data["Bull Short Return "+str(cost*100)+"%"]=short_daily_return(data["Bull Return"],cost)
    data["Bear Short Return "+str(cost*100)+"%"]=short_daily_return(data["Bear Return"],cost)
    
    # Calculating cumulative return
    data["Index CuRe"]=Cumulative_Return(data["Index Return"],data[period])
    data["Bull Short CuRe "+str(cost*100)+"%"]=Cumulative_Return(data["Bull Short Return "+str(cost*100)+"%"],data[period])
    data["Bear Short CuRe "+str(cost*100)+"%"]=Cumulative_Return(data["Bear Short Return "+str(cost*100)+"%"],data[period])
    
    # Calculating Sharpe ratio
    data["Index Sharpe Ratio"]=Sharpe_Ratio(data["Index Return"],data["Rf"],data[period])
    data["Bull Short Sharpe Ratio "+str(cost*100)+"%"]=Sharpe_Ratio(data["Bull Short Return "+str(cost*100)+"%"],data["Rf"],data[period])
    data["Bear Short Sharpe Ratio "+str(cost*100)+"%"]=Sharpe_Ratio(data["Bear Short Return "+str(cost*100)+"%"],data["Rf"],data[period])
    
    # Calculating Sortino ratio
    data["Index Sortino Ratio"]=Sortino_Ratio(data["Index Return"],data["Rf"],data[period])
    data["Bull Short Sortino Ratio "+str(cost*100)+"%"]=Sortino_Ratio(data["Bull Short Return "+str(cost*100)+"%"],data["Rf"],data[period])
    data["Bear Short Sortino Ratio "+str(cost*100)+"%"]=Sortino_Ratio(data["Bear Short Return "+str(cost*100)+"%"],data["Rf"],data[period])
    
    # Calculating Analytical VaR
    data["Index Analytical VaR"]=Analytical_VaR(data["Index Return"],data[period])
    data["Bull Short Analytical VaR "+str(cost*100)+"%"]=Analytical_VaR(data["Bull Short Return "+str(cost*100)+"%"],data[period])
    data["Bear Short Analytical VaR "+str(cost*100)+"%"]=Analytical_VaR(data["Bear Short Return "+str(cost*100)+"%"],data[period])
    
    # Calculating VaR
    data["Index VaR"]=VaR(data["Index Return"],data[period])
    data["Bull Short VaR "+str(cost*100)+"%"]=VaR(data["Bull Short Return "+str(cost*100)+"%"],data[period])
    data["Bear Short VaR "+str(cost*100)+"%"]=VaR(data["Bear Short Return "+str(cost*100)+"%"],data[period])
    
    # Calculating Expected VaR
    data["Index Expected VaR"]=Expected_VaR(data["Index Return"],data[period])
    data["Bull Short Expected VaR "+str(cost*100)+"%"]=Expected_VaR(data["Bull Short Return "+str(cost*100)+"%"],data[period])
    data["Bear Short Expected VaR "+str(cost*100)+"%"]=Expected_VaR(data["Bear Short Return "+str(cost*100)+"%"],data[period])
    
    # Calculating Best strategy using cumulative return results
    data["CuRe-Portfolio Short 1:0 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],1,0)
    data["CuRe-Portfolio Short 0.75:0.25 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],0.75,0.25)
    data["CuRe-Portfolio Short 0.67:0.33 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],0.67,0.33)
    data["CuRe-Portfolio Short 0.5:0.5 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],0.5,0.5)
    data["CuRe-Portfolio Short 0.33:0.67 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],0.33,0.67)
    data["CuRe-Portfolio Short 0.25:0.75 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],0.25,0.75)
    data["CuRe-Portfolio Short 0:1 "+str(cost*100)+"%"]=Portfolio(data["Bull Short CuRe "+str(cost*100)+"%"],data["Bear Short CuRe "+str(cost*100)+"%"],0,1)
    
    # Calculating Best strategy using Sharpe ratio results
    data["Sharpe Ratio-Portfolio Short 1:0 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],1,0)
    data["Sharpe Ratio-Portfolio Short 0.75:0.25 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],0.75,0.25)
    data["Sharpe Ratio-Portfolio Short 0.67:0.33 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],0.67,0.33)
    data["Sharpe Ratio-Portfolio Short 0.5:0.5 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],0.5,0.5)
    data["Sharpe Ratio-Portfolio Short 0.33:0.67 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],0.33,0.67)
    data["Sharpe Ratio-Portfolio Short 0.25:0.75 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],0.25,0.75)
    data["Sharpe Ratio-Portfolio Short 0:1 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sharpe Ratio "+str(cost*100)+"%"],data["Bear Short Sharpe Ratio "+str(cost*100)+"%"],0,1)
    
    # Calculating Best strategy using Sortino ratio results
    data["Sortino Ratio-Portfolio Short 1:0 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],1,0)
    data["Sortino Ratio-Portfolio Short 0.75:0.25 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],0.75,0.25)
    data["Sortino Ratio-Portfolio Short 0.67:0.33 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],0.67,0.33)
    data["Sortino Ratio-Portfolio Short 0.5:0.5 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],0.5,0.5)
    data["Sortino Ratio-Portfolio Short 0.33:0.67 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],0.33,0.67)
    data["Sortino Ratio-Portfolio Short 0.25:0.75 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],0.25,0.75)
    data["Sortino Ratio-Portfolio Short 0:1 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Sortino Ratio "+str(cost*100)+"%"],data["Bear Short Sortino Ratio "+str(cost*100)+"%"],0,1)
    
    # Calculating Best strategy using expected VaR results
    data["Expected VaR-Portfolio Short 1:0 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],1,0)
    data["Expected VaR-Portfolio Short 0.75:0.25 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],0.75,0.25)
    data["Expected VaR-Portfolio Short 0.67:0.33 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],0.67,0.33)
    data["Expected VaR-Portfolio Short 0.5:0.5 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],0.5,0.5)
    data["Expected VaR-Portfolio Short 0.33:0.67 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],0.33,0.67)
    data["Expected VaR-Portfolio Short 0.25:0.75 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],0.25,0.75)
    data["Expected VaR-Portfolio Short 0:1 "+str(cost*100)+"%"]=Portfolio(data["Bull Short Expected VaR "+str(cost*100)+"%"],data["Bear Short Expected VaR "+str(cost*100)+"%"],0,1)

### Creating two datasets: quarterly analysis and annual analysis

In [15]:
data_quarter=data.copy()
data_yearly=data.copy()

### Calculating all portfolios using different shorting costs

##### There are three measurments cumulative return, Sharpe ratio and Sortino ratio

##### Cumulative return:
for quarterly data
at 15% cost : the difference is not significant in 5% but it is significant in 10% (at 14% the cost is significant at 5%)
at 16% cost: the difference is not significant at 10% anymore (it is significant for 15%)
at 21% cost: the absolute difference becomes negative (at 20% it is positive).
For year the values 16% (not significant at 95% but significant at 90%), 17% (not significant in 90% anymore), 21% cost the absolute difference becomes negative.

In [16]:
Main_Function(data_quarter, 0.14, "Quarter")
Main_Function(data_quarter, 0.15, "Quarter")
Main_Function(data_quarter, 0.20, "Quarter")

In [17]:
Main_Function(data_yearly, 0.15, "Year")
Main_Function(data_yearly, 0.16, "Year")
Main_Function(data_yearly, 0.20, "Year")

#### Sharpe ratio : 
for quarterly data
at 0 cost : the difference is not significant in 95% but it is significant in 90% (p-value is 0.08)
at 0.2% cost: the difference is not significant at 90% anymore (it is significant for 0.1% at 90% p-value is 0.098)
at 4% cost: the absolute difference becomes negative (3% is positive).
For yearly data, the differnece is not significant at 0 cost (not in 95% or 90%). The absolute difference becomse negative at 5% cost (at 4% it is positive).

In [18]:
Main_Function(data_quarter, 0, "Quarter")
Main_Function(data_quarter, 0.001, "Quarter")
Main_Function(data_quarter, 0.03, "Quarter")

In [19]:
Main_Function(data_yearly, 0.04, "Year")

###### Sortino ratio : 
for quarterly data
at 0.7% cost : the difference is not significant in 95% but it is significant in 10% (at 0.6% the cost is significant at 95%)
at 2% cost: the difference is not significant at 10% anymore (it is significant in 1%)
at 6% cost: the absolute difference becomes negative (5% is positive but so small)
For yearly data at zero cost it is significant at 90% of consfidence. It is not significant at 95% when the cost increases to 0.006. The absolute difference becomes negative at 5% cost.

In [20]:
Main_Function(data_quarter, 0.006, "Quarter")
Main_Function(data_quarter, 0.01, "Quarter")
Main_Function(data_quarter, 0.05, "Quarter")

In [21]:
Main_Function(data_yearly, 0, "Year")
Main_Function(data_yearly, 0.005, "Year")
Main_Function(data_yearly, 0.04, "Year")

In [22]:
data_quarter=Remove_Character(data_quarter, data_quarter["Sortino Ratio-Portfolio Short 0:1 5.0%"], ".")
data_yearly=Remove_Character(data_yearly, data_yearly["Sortino Ratio-Portfolio Short 0:1 4.0%"], ".")

### Comparing the means of portfolios with indices

In [23]:
def print_means_difference (dataset, investment, measurment, cost):
    Name = measurment +"-Portfolio "+investment
    print(
        np.mean(dataset[Name +" 1:0 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        np.mean(dataset[Name + " 0.75:0.25 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        np.mean(dataset[Name + " 0.67:0.33 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        np.mean(dataset[Name + " 0.5:0.5 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        np.mean(dataset[Name + " 0.33:0.67 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        np.mean(dataset[Name + " 0.25:0.75 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        np.mean(dataset[Name +" 0:1 "+str(cost*100)+"%"])- np.mean(dataset["Index "+measurment]),
        sep='\n')

In [24]:
print_means_difference(data_quarter,"Short","CuRe",0.14)

-0.11473828328322622
-0.08269265441507026
-0.07243805317726018
-0.05064702554691418
-0.028855997916568062
-0.018601396678758125
0.013444232189397935


In [25]:
print_means_difference(data_quarter,"Short","CuRe",0.15)

-0.11718481011836024
-0.08506864614501639
-0.0747914736735465
-0.05295248217167277
-0.031113490669798988
-0.02083631819832899
0.011279845775014778


In [26]:
print_means_difference(data_quarter,"Short","CuRe",0.20)

-0.12918560521482292
-0.09672349218125653
-0.08633561601051525
-0.06426137914768998
-0.042187142284864795
-0.031799266114123544
0.0006628469194429124


In [27]:
print_means_difference(data_yearly,"Short","CuRe",0.15)

-0.4818004953947907
-0.35048883625923566
-0.308469105335858
-0.21917717712368068
-0.12988524891150327
-0.08786551798812568
0.043446141147429285


In [28]:
print_means_difference(data_yearly,"Short","CuRe",0.16)

-0.4941742499024258
-0.361694037570219
-0.3193003696239126
-0.22921382523801193
-0.13912728085211118
-0.09673361290580497
0.03574659942640196


In [29]:
print_means_difference(data_yearly,"Short","CuRe",0.20)

-0.5436972185080048
-0.40654046537797184
-0.3626503043763612
-0.26938371224793867
-0.17611712011951586
-0.1322269591179052
0.004929794012128105


In [30]:
print_means_difference(data_quarter,"Short","Sharpe Ratio",0.001)

-0.1052960351619063
-0.07717909257723508
-0.0681816709501403
-0.049062149992563864
-0.029942629034987435
-0.020945207407892658
0.0071717351767785695


In [31]:
print_means_difference(data_quarter,"Short","Sharpe Ratio",0.03)

-0.11095794506534287
-0.08284343831984772
-0.07384679616128934
-0.05472893157435269
-0.035611066987416035
-0.02661442482885767
0.001500081916637544


In [32]:
print_means_difference(data_yearly,"Short","Sharpe Ratio",0.04)

-0.08788011040707067
-0.065856581255307
-0.05880905192674263
-0.043833052103543324
-0.028857052280344027
-0.021809522951779636
0.00021400619998403203


In [33]:
print_means_difference(data_quarter,"Short","Sortino Ratio",0.006)

-0.1071350643693207
-0.07760395667772428
-0.0681540022164134
-0.04807284898612779
-0.027991695755842184
-0.018541741294531344
0.010989366397065152


In [48]:
print_means_difference(data_quarter,"Short","Sortino Ratio",0.01)

-0.10912676660266936
-0.07965916562978714
-0.07022953331846483
-0.05019156465690487
-0.03015359599534487
-0.02072396368402258
0.008743637288859706


In [49]:
print_means_difference(data_quarter,"Short","Sortino Ratio",0.05)

-0.11688135511904035
-0.08763248635445889
-0.07827284834979278
-0.058383617589877326
-0.03849438682996184
-0.029134748825295768
0.00011411993928581166


In [50]:
print_means_difference(data_yearly,"Short","Sortino Ratio",0.005)

-0.08260899226072384
-0.059974142775838846
-0.05273099094067567
-0.03733929329095386
-0.02194759564123207
-0.014704443806068876
0.007930405678816095


In [51]:
print_means_difference(data_yearly,"Short","Sortino Ratio",0.04)

-0.08901233819512194
-0.06643850365666085
-0.059214876604353345
-0.04386466911819983
-0.02851446163204631
-0.021290834579738797
0.0012829999587222266


### Statistically comparing the means of portfolios with indices

In [34]:
def print_compare_means (dataset, investment, measurment, cost):
    Name = measurment +"-Portfolio "+investment
    print(
        st.ttest_ind(dataset[Name +" 1:0 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        st.ttest_ind(dataset[Name + " 0.75:0.25 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        st.ttest_ind(dataset[Name + " 0.67:0.33 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        st.ttest_ind(dataset[Name + " 0.5:0.5 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        st.ttest_ind(dataset[Name + " 0.33:0.67 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        st.ttest_ind(dataset[Name + " 0.25:0.75 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        st.ttest_ind(dataset[Name + " 0:1 "+str(cost*100)+"%"].to_numpy(), dataset["Index "+measurment].to_numpy(), equal_var=True),
        sep='\n')

In [35]:
print_compare_means(data_quarter,"Short","CuRe",0.14)

Ttest_indResult(statistic=-17.228972442079083, pvalue=1.1786823319610528e-63)
Ttest_indResult(statistic=-19.680057243382528, pvalue=1.9473241023499703e-81)
Ttest_indResult(statistic=-20.326558637278257, pvalue=1.9542337344893415e-86)
Ttest_indResult(statistic=-17.969438097557997, pvalue=7.940149098956482e-69)
Ttest_indResult(statistic=-8.68423216041974, pvalue=6.002914081852266e-18)
Ttest_indResult(statistic=-4.768682185649574, pvalue=1.938072288380017e-06)
Ttest_indResult(statistic=2.1399192701989533, pvalue=0.03243751489740063)


In [36]:
print_compare_means(data_quarter,"Short","CuRe",0.15)

Ttest_indResult(statistic=-17.563023612881647, pvalue=5.758223796608115e-66)
Ttest_indResult(statistic=-20.21665200788841, pvalue=1.4108684414690457e-85)
Ttest_indResult(statistic=-20.96333621381971, pvalue=1.767750867960772e-91)
Ttest_indResult(statistic=-18.778167974446554, pvalue=1.1255250351848301e-74)
Ttest_indResult(statistic=-9.354373837714732, pvalue=1.543519643317034e-20)
Ttest_indResult(statistic=-5.334534016695113, pvalue=1.0252831253837828e-07)
Ttest_indResult(statistic=1.792045391642872, pvalue=0.07322115182238112)


In [37]:
print_compare_means(data_quarter,"Short","CuRe",0.20)

Ttest_indResult(statistic=-19.183332842197544, pvalue=1.1078169834884772e-77)
Ttest_indResult(statistic=-22.82644035792562, pvalue=6.796243823144954e-107)
Ttest_indResult(statistic=-24.065867081370524, pvalue=1.1489737408124963e-117)
Ttest_indResult(statistic=-22.733439015489598, pvalue=4.210142864632363e-106)
Ttest_indResult(statistic=-12.622297576483705, pvalue=1.1341736421138794e-35)
Ttest_indResult(statistic=-8.088431599317317, pvalue=8.521388638872793e-16)
Ttest_indResult(statistic=0.10434624744985417, pvalue=0.9169011875236561)


In [38]:
print_compare_means(data_yearly,"Short","CuRe",0.15)

Ttest_indResult(statistic=-16.90901107889538, pvalue=5.619768831764018e-55)
Ttest_indResult(statistic=-18.36314427675152, pvalue=5.43639294356943e-63)
Ttest_indResult(statistic=-18.747326446316343, pvalue=3.7406256952514274e-65)
Ttest_indResult(statistic=-17.784661954191844, pvalue=9.06399564825229e-60)
Ttest_indResult(statistic=-10.908527727123849, pvalue=6.632281649486203e-26)
Ttest_indResult(statistic=-6.648799542918358, pvalue=5.5108086120644885e-11)
Ttest_indResult(statistic=2.0888889476797172, pvalue=0.03703723357597978)


In [39]:
print_compare_means(data_yearly,"Short","CuRe",0.16)

Ttest_indResult(statistic=-17.203283966177693, pvalue=1.4163784698855559e-56)
Ttest_indResult(statistic=-18.814673177967506, pvalue=1.5563978250598334e-65)
Ttest_indResult(statistic=-19.277972304365264, pvalue=3.6200059003002627e-68)
Ttest_indResult(statistic=-18.509355036625987, pvalue=8.211316909567908e-64)
Ttest_indResult(statistic=-11.631504679463184, pvalue=5.622609671970557e-29)
Ttest_indResult(statistic=-7.280539498164117, pvalue=8.043711156452217e-13)
Ttest_indResult(statistic=1.705900266746679, pvalue=0.0884197087822546)


In [40]:
print_compare_means(data_yearly,"Short","CuRe",0.20)

Ttest_indResult(statistic=-18.333998200777163, pvalue=7.918519159909384e-63)
Ttest_indResult(statistic=-20.55674032507444, pvalue=1.5066902151748566e-75)
Ttest_indResult(statistic=-21.330816703532065, pvalue=4.3743320172295304e-80)
Ttest_indResult(statistic=-21.3373125327148, pvalue=4.005370835393158e-80)
Ttest_indResult(statistic=-14.45769050750495, pvalue=3.316668216860867e-42)
Ttest_indResult(statistic=-9.740779943504805, pvalue=2.993753834456793e-21)
Ttest_indResult(statistic=0.22843078117094623, pvalue=0.8193704767375394)


In [41]:
print_compare_means(data_quarter,"Short","Sharpe Ratio",0.001)

Ttest_indResult(statistic=-24.214244023645872, pvalue=5.550236072705431e-119)
Ttest_indResult(statistic=-22.372605797708548, pvalue=4.734415085861328e-103)
Ttest_indResult(statistic=-20.900127826585877, pvalue=5.666489250068481e-91)
Ttest_indResult(statistic=-15.872844562119111, pvalue=1.186851814186163e-54)
Ttest_indResult(statistic=-9.187481945765619, pvalue=7.08189009667499e-20)
Ttest_indResult(statistic=-6.079404121491263, pvalue=1.3503002290223804e-09)
Ttest_indResult(statistic=1.6519143919487143, pvalue=0.09865116918772526)


In [42]:
print_compare_means(data_quarter,"Short","Sharpe Ratio",0.03)

Ttest_indResult(statistic=-25.438372327280618, pvalue=4.7816360189331785e-130)
Ttest_indResult(statistic=-23.950612494850006, pvalue=1.1986673673254566e-116)
Ttest_indResult(statistic=-22.586743860273543, pvalue=7.394154901399464e-105)
Ttest_indResult(statistic=-17.6947718529784, pvalue=6.899781486822199e-67)
Ttest_indResult(statistic=-10.938084931658636, pvalue=2.3251756457366782e-27)
Ttest_indResult(statistic=-7.737291313969879, pvalue=1.3555804716646316e-14)
Ttest_indResult(statistic=0.3463263389829757, pvalue=0.7291204900176285)


In [43]:
print_compare_means(data_yearly,"Short","Sharpe Ratio",0.04)

Ttest_indResult(statistic=-18.8507034652742, pvalue=9.731291617628294e-66)
Ttest_indResult(statistic=-17.849422880839477, pvalue=3.969498428781813e-60)
Ttest_indResult(statistic=-16.882648009666145, pvalue=7.803867420064912e-55)
Ttest_indResult(statistic=-13.341498392109234, pvalue=9.141949942169718e-37)
Ttest_indResult(statistic=-8.363107884731967, pvalue=2.7500208103058815e-16)
Ttest_indResult(statistic=-5.985315302180512, pvalue=3.2744234169615355e-09)
Ttest_indResult(statistic=0.04663044331614151, pvalue=0.9628195530250037)


In [52]:
print_compare_means(data_quarter,"Short","Sortino Ratio",0.006)

Ttest_indResult(statistic=-23.54517792458238, pvalue=4.307105712062726e-113)
Ttest_indResult(statistic=-21.3726213256963, pvalue=8.79059678517838e-95)
Ttest_indResult(statistic=-19.761443956894944, pvalue=4.645822753744505e-82)
Ttest_indResult(statistic=-14.56797986847694, pvalue=1.3967712223842526e-46)
Ttest_indResult(statistic=-8.085499203810015, pvalue=8.72470364821661e-16)
Ttest_indResult(statistic=-5.166212940755753, pvalue=2.5367472872767205e-07)
Ttest_indResult(statistic=1.988228413399132, pvalue=0.0468724095011657)


In [53]:
print_compare_means(data_quarter,"Short","Sortino Ratio",0.01)

Ttest_indResult(statistic=-23.721529637473733, pvalue=1.2383296462329638e-114)
Ttest_indResult(statistic=-21.593822591158222, pvalue=1.3768943864546818e-96)
Ttest_indResult(statistic=-19.996494479529627, pvalue=7.218436098970524e-84)
Ttest_indResult(statistic=-14.820606309291989, pvalue=4.255657456306416e-48)
Ttest_indResult(statistic=-8.32880171668978, pvalue=1.2004973564081437e-16)
Ttest_indResult(statistic=-5.397047253600479, pvalue=7.27391347766537e-08)
Ttest_indResult(statistic=1.8061713919857678, pvalue=0.07098649947454448)


In [54]:
print_compare_means(data_quarter,"Short","Sortino Ratio",0.05)

Ttest_indResult(statistic=-25.43285567841415, pvalue=5.373463458018023e-130)
Ttest_indResult(statistic=-23.73759005053918, pvalue=8.955042145704404e-115)
Ttest_indResult(statistic=-22.27442062955142, pvalue=3.157590414159794e-102)
Ttest_indResult(statistic=-17.27244815561532, pvalue=5.924796061540631e-64)
Ttest_indResult(statistic=-10.697899212008744, pvalue=2.909419096611678e-26)
Ttest_indResult(statistic=-7.648421737487747, pvalue=2.68136249522298e-14)
Ttest_indResult(statistic=0.023856140841234524, pvalue=0.9809688622585997)


In [55]:
print_compare_means(data_yearly,"Short","Sortino Ratio",0.005)

Ttest_indResult(statistic=-17.514589902577136, pvalue=2.7972267159010944e-58)
Ttest_indResult(statistic=-16.181027559308774, pvalue=4.448690401201176e-51)
Ttest_indResult(statistic=-15.08310410529649, pvalue=2.3066558406774204e-45)
Ttest_indResult(statistic=-11.286442165686054, pvalue=1.7116100750230305e-27)
Ttest_indResult(statistic=-6.2552329973026675, pvalue=6.498337554237039e-10)
Ttest_indResult(statistic=-3.9485813695623837, pvalue=8.56325804967621e-05)
Ttest_indResult(statistic=1.6716274813516634, pvalue=0.09499381396399237)


In [56]:
print_compare_means(data_yearly,"Short","Sortino Ratio",0.04)

Ttest_indResult(statistic=-18.805500095600426, pvalue=1.753967882906587e-65)
Ttest_indResult(statistic=-17.839376364666474, pvalue=4.512316041706401e-60)
Ttest_indResult(statistic=-16.866068065356195, pvalue=9.592548900659056e-55)
Ttest_indResult(statistic=-13.25168842634653, pvalue=2.4388350903543636e-36)
Ttest_indResult(statistic=-8.16817042005832, pvalue=1.2375093209395702e-15)
Ttest_indResult(statistic=-5.75963402804656, pvalue=1.2071734442247572e-08)
Ttest_indResult(statistic=0.2735792288121754, pvalue=0.7844795065777931)
